In [ ]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [ ]:
!nvidia-smi

Mon Nov 29 13:44:11 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
cd /content/gdrive/MyDrive/Aspect-Term-Extraction-and-Analysis/data/

/content/gdrive/MyDrive/Aspect-Term-Extraction-and-Analysis/data


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 59 kB 6.8 MB/s 
     |████████████████████████████████| 3.3 MB 28.2 MB/s 
     |████████████████████████████████| 895 kB 30.0 MB/s 
     |████████████████████████████████| 596 kB 40.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics
import torch
from transformers import BertModel,BertTokenizer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score

In [ ]:
class ABSA(torch.nn.Module):
    def __init__(self, pretrain_model):
        super(ABSA, self).__init__()
        self.bert = BertModel.from_pretrained(pretrain_model)
        self.cnn = torch.nn.Conv1d(self.bert.config.hidden_size, 3, 1)

    def forward(self, idt, labt, masks, segt):
        _, pooled_outputs = self.bert(input_ids=idt, attention_mask=masks, token_type_ids=segt, return_dict = False)
        cnn_outputs = self.cnn(pooled_outputs.view(pooled_outputs.shape[0], pooled_outputs.shape[1], 1))
        if labt is not None:
            loss = torch.nn.CrossEntropyLoss()(cnn_outputs, labt.view(labt.shape[0], 1))
            return loss
        else:
            return cnn_outputs

In [ ]:
class dataset(Dataset):
    def __init__(self, df, tokenizer):
        self.df = df
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        tokens, tags, polarity = self.df.iloc[idx, :3].values
        tokens,tags, polarity = tokens.replace("'", "").strip("][").split(', '),tags.strip('][').split(', '),polarity.strip('][').split(', ')
        bert_tokens,bert_att,polarity_label = [],[],0
        stop=len(tokens)
        i=0
        while(i<stop):
            tok=tokens[i]
            t = self.tokenizer.tokenize(tok)
            bert_tokens += t
            if int(polarity[i]) != -1:
                bert_att += t
                polarity_label = int(polarity[i])
            i+=1
        tok_len=len(bert_tokens)
        at_len=len(bert_att)
        segment_tensor = [0]*(tok_len+1) + [1]*(at_len+1)
        bert_tokens = ['[cls]'] + bert_tokens + ['[sep]'] + bert_att
        ids_tensor,polarity_tensor,segment_tensor = torch.tensor(self.tokenizer.convert_tokens_to_ids(bert_tokens)),torch.tensor(polarity_label),torch.tensor(segment_tensor)

        return bert_tokens, ids_tensor, segment_tensor, polarity_tensor

    def __len__(self):
        return len(self.df)

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = ABSA("bert-base-uncased").to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
samplez1=1500
samplez2=1000

restaurants_train, restaurants_test = dataset(pd.read_csv("restaurants_train.csv")[:samplez1], tokenizer), dataset(pd.read_csv("restaurants_test.csv")[:samplez1], tokenizer)
laptops_train, laptops_test = dataset(pd.read_csv("laptops_train.csv")[:samplez2], tokenizer), dataset(pd.read_csv("laptops_test.csv")[:samplez2], tokenizer)
train, test = ConcatDataset([laptops_train, restaurants_train]),  ConcatDataset([laptops_test, restaurants_test])

In [ ]:
def mini_batch(samples):
    idt = pad_sequence([s[1] for s in samples], batch_first=True)
    seg= pad_sequence([s[2] for s in samples], batch_first=True)
    label_ids = torch.stack([s[3] for s in samples])
    masks = torch.zeros(idt.shape, dtype=torch.long).masked_fill(idt != 0, 1)
    return idt, seg, masks, label_ids

In [ ]:
train_loader = DataLoader(train, batch_size=50, collate_fn=mini_batch, shuffle = True)
test_loader = DataLoader(test, batch_size=200, collate_fn=mini_batch, shuffle = True)

In [ ]:
def train_model(loader, epochs):
    all_data = len(loader)
    for epoch in range(epochs):
        lv,losses,correct_predictions = 0,[],0
        for data in loader:
            idt, segt, masks, label_ids = data
            idt,segt,label_ids,masks = idt.to(DEVICE),segt.to(DEVICE),label_ids.to(DEVICE),masks.to(DEVICE)
            loss = model(idt=idt, labt=label_ids, masks=masks, segt=segt)
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            lv += 1
            print('Epoch:', epoch, "\t loss:", np.mean(losses), "\t Batch:", lv, "/" , all_data)
        

In [ ]:
train_model(train_loader, 6)

Epoch: 0 	 loss: 0.4587206542491913 	 Batch: 1 / 50
Epoch: 0 	 loss: 0.4208887815475464 	 Batch: 2 / 50
Epoch: 0 	 loss: 0.4192277789115906 	 Batch: 3 / 50
Epoch: 0 	 loss: 0.4137266278266907 	 Batch: 4 / 50
Epoch: 0 	 loss: 0.3905121684074402 	 Batch: 5 / 50
Epoch: 0 	 loss: 0.39265768726666767 	 Batch: 6 / 50
Epoch: 0 	 loss: 0.4006036009107317 	 Batch: 7 / 50
Epoch: 0 	 loss: 0.4302043318748474 	 Batch: 8 / 50
Epoch: 0 	 loss: 0.4210501280095842 	 Batch: 9 / 50
Epoch: 0 	 loss: 0.4160179942846298 	 Batch: 10 / 50
Epoch: 0 	 loss: 0.42520744421265344 	 Batch: 11 / 50
Epoch: 0 	 loss: 0.4251002346475919 	 Batch: 12 / 50
Epoch: 0 	 loss: 0.41790406979047334 	 Batch: 13 / 50
Epoch: 0 	 loss: 0.4254915841988155 	 Batch: 14 / 50
Epoch: 0 	 loss: 0.4295529961585999 	 Batch: 15 / 50
Epoch: 0 	 loss: 0.43131424859166145 	 Batch: 16 / 50
Epoch: 0 	 loss: 0.4253312542158015 	 Batch: 17 / 50
Epoch: 0 	 loss: 0.42610444459650254 	 Batch: 18 / 50
Epoch: 0 	 loss: 0.41843577435142115 	 Batch: 19 /

KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), '/content/gdrive/My Drive/Aspect-Term-Extraction-and-Analysis/data/AscpectBasedSentimentAnalysis.pkl')

In [ ]:
pred , y = [] , []
with torch.no_grad():
    for i in test_loader:
        idt, segt, masks, label_ids = i
        idt, segt, masks = idt.to(DEVICE),segt.to(DEVICE), masks.to(DEVICE)
        _, preds = torch.max(model(idt=idt, labt=None, masks =masks, segt=segt), dim=1)
        pred += list([int(pr) for pr in preds])
        y += list([int(li) for li in label_ids])

print("Accuracy is "+str(accuracy_score(pred, y)))

Accuracy is 0.7749287749287749


In [ ]:
import os
os.getcwd()

'/content/gdrive/My Drive/Aspect-Term-Extraction-and-Analysis/data'

In [ ]:
PATH='/content/gdrive/My Drive/Aspect-Term-Extraction-and-Analysis/data/AscpectBasedSentimentAnalysis.pkl'
model = ABSA("bert-base-uncased")
model.load_state_dict(torch.load(PATH))
model = model.to(DEVICE)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
pred , y = [] , []
with torch.no_grad():
    for i in test_loader:
        idt, segt, masks, label_ids = i
        idt, segt, masks = idt.to(DEVICE),segt.to(DEVICE), masks.to(DEVICE)
        _, preds = torch.max(model(idt=idt, labt=None, masks =masks, segt=segt), dim=1)
        pred += list([int(pr) for pr in preds])
        y += list([int(li) for li in label_ids])

print("Accuracy is "+str(accuracy_score(pred, y)))

Accuracy is 0.7749287749287749
